<a href="https://colab.research.google.com/github/DlKrone/OOTDiffusion/blob/main/Z_Image_Turbo_jupyter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/comfyanonymous/ComfyUI

%cd /content/ComfyUI
!pip install -r requirements.txt

!apt -y install -qq aria2

!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/T5B/Z-Image-Turbo-FP8/resolve/main/z-image-turbo-fp8-e4m3fn.safetensors -d /content/ComfyUI/models/diffusion_models -o z-image-turbo-fp8-e4m3fn.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/text_encoders/qwen_3_4b.safetensors -d /content/ComfyUI/models/clip -o qwen_3_4b.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Comfy-Org/z_image_turbo/resolve/main/split_files/vae/ae.safetensors -d /content/ComfyUI/models/vae -o ae.safetensors

fatal: destination path 'ComfyUI' already exists and is not an empty directory.
/content/ComfyUI
aria2 is already the newest version (1.36.0-1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
763c58|OK  |       0B/s|/content/ComfyUI/models/diffusion_models/z-image-turbo-fp8-e4m3fn.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
9a8b6a|OK  |       0B/s|/content/ComfyUI/models/clip/qwen_3_4b.safetensors

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
f30202|OK  |       0B/s|/content/ComfyUI/models/vae/ae.safetensors

Status Legend:
(OK):download completed.


In [2]:
%cd /content/ComfyUI

import os, random, time

import torch
import numpy as np
from PIL import Image

from nodes import NODE_CLASS_MAPPINGS

UNETLoader = NODE_CLASS_MAPPINGS["UNETLoader"]()
CLIPLoader = NODE_CLASS_MAPPINGS["CLIPLoader"]()
VAELoader = NODE_CLASS_MAPPINGS["VAELoader"]()
CLIPTextEncode = NODE_CLASS_MAPPINGS["CLIPTextEncode"]()
KSampler = NODE_CLASS_MAPPINGS["KSampler"]()
VAEDecode = NODE_CLASS_MAPPINGS["VAEDecode"]()
EmptyLatentImage = NODE_CLASS_MAPPINGS["EmptyLatentImage"]()

with torch.inference_mode():
    unet = UNETLoader.load_unet("z-image-turbo-fp8-e4m3fn.safetensors", "fp8_e4m3fn_fast")[0]
    clip = CLIPLoader.load_clip("qwen_3_4b.safetensors", type="lumina2")[0]
    vae = VAELoader.load_vae("ae.safetensors")[0]

@torch.inference_mode()
def generate(input):
    tmp_dir="/content/ComfyUI/output"
    os.makedirs(tmp_dir, exist_ok=True)

    values = input["input"]

    positive_prompt = values['positive_prompt']
    negative_prompt = values['negative_prompt']
    seed = values['seed'] # 0
    steps = values['steps'] # 9
    cfg = values['cfg'] # 1.0
    sampler_name = values['sampler_name'] # euler
    scheduler = values['scheduler'] # simple
    denoise = values['denoise'] # 1.0
    width = values['width'] # 1024
    height = values['height'] # 1024
    batch_size = values['batch_size'] # 1.0

    if seed == 0:
        random.seed(int(time.time()))
        seed = random.randint(0, 18446744073709551615)

    positive = CLIPTextEncode.encode(clip, positive_prompt)[0]
    negative = CLIPTextEncode.encode(clip, negative_prompt)[0]
    latent_image = EmptyLatentImage.generate(width, height, batch_size=batch_size)[0]
    samples = KSampler.sample(unet, seed, steps, cfg, sampler_name, scheduler, positive, negative, latent_image, denoise=denoise)[0]
    decoded = VAEDecode.decode(vae, samples)[0].detach()
    Image.fromarray(np.array(decoded*255, dtype=np.uint8)[0]).save(f"{tmp_dir}/z_image_turbo.png")

    result = f"{tmp_dir}/z_image_turbo.png"

    return result

/content/ComfyUI


In [4]:
import gradio as gr
from PIL import Image

# Функция-обертка для обработки данных из интерфейса
def run_generation(positive_prompt, negative_prompt, use_negative, size, seed, steps, cfg):
    # Превращаем строку "1280x720" в числа width и height
    width, height = map(int, size.split("x"))

    # Если галочка выключена, отправляем пустую строку вместо негативного промпта
    final_negative = negative_prompt if use_negative else ""

    # Формируем словарь, который ожидает твоя функция generate
    payload = {
        "input": {
            "positive_prompt": positive_prompt,
            "negative_prompt": final_negative,
            "width": width,
            "height": height,
            "batch_size": 1,
            "seed": int(seed),
            "steps": int(steps),
            "cfg": float(cfg),
            "sampler_name": "euler",
            "scheduler": "simple",
            "denoise": 1.0,
        }
    }

    # Вызываем твою основную функцию генерации
    # (Убедись, что функция generate определена выше в твоем коде)
    output = generate(payload)
    return Image.open(output)

# Создание интерфейса Gradio
with gr.Blocks(title="AI Image Generator") as demo:
    gr.Markdown("# 🎨 Генератор изображений")

    with gr.Row():
        # Левая колонка: Настройки
        with gr.Column():
            pos = gr.Textbox(
                label="Positive Prompt (Что рисуем)",
                lines=3,
                placeholder="Например: A foggy forest with a shadow creature..."
            )

            with gr.Row():
                use_neg = gr.Checkbox(label="Включить Negative Prompt", value=True)
                size = gr.Dropdown(
                    label="Разрешение",
                    choices=[
                        "1024x1024", # Квадрат
                        "1280x720",  # 16:9 (Горизонтальный)
                        "720x1280",  # 9:16 (Вертикальный)
                        "512x512",   # Маленький квадрат
                        "1216x832",  # Широкий
                        "832x1216"   # Высокий
                    ],
                    value="1024x1024"
                )

            neg = gr.Textbox(
                label="Negative Prompt (Чего НЕ должно быть)",
                lines=2,
                value="photo, realistic, 3d render, watermark"
            )

            # Логика: скрыть/показать поле негативного промпта при нажатии на чекбокс
            use_neg.change(lambda x: gr.update(visible=x), inputs=use_neg, outputs=neg)

            with gr.Row():
                seed = gr.Number(label="Seed (Число для случайности)", value=0, precision=0)

            steps = gr.Slider(minimum=1, maximum=50, step=1, label="Steps (Качество/Детали)", value=9)
            cfg = gr.Slider(minimum=1, maximum=20, step=0.5, label="CFG Scale (Следование тексту)", value=1.0)

            btn = gr.Button("🚀 ГЕНЕРИРОВАТЬ", variant="primary")

        # Правая колонка: Результат
        with gr.Column():
            result_img = gr.Image(label="Результат", type="pil")

    # Связываем кнопку с функцией
    btn.click(
        fn=run_generation,
        inputs=[pos, neg, use_neg, size, seed, steps, cfg],
        outputs=result_img
    )

# Запуск приложения
if __name__ == "__main__":
    demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9bf955899acd4cfafe.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
